# Convert suite2p output data

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import flammkuchen as fl
from pathlib import Path
from tifffile import imwrite

In [ ]:
def save_anatomy(dest, stack):
    """ Utility function to save stack of consisten histogram rescaling
    and metadata formatting required by CMTK.
    """
    norm = stack - np.percentile(stack, 1)
    norm[norm < 0] = 0
    norm = norm / np.percentile(norm, 99.9)
    norm[norm > 1] = 1
    
    norm *= 255
    
    imwrite(str(dest), norm[::-1, :, :].astype(np.uint8), resolution=(1/0.6, 1/0.6))
    
def save_rois(dest, stack):
    """ Utility function to save stack of consisten histogram rescaling
    and metadata formatting required by CMTK.
    """
    
    # Check for funny values in the stack:
    if (stack < 0).sum() > 0 or (stack > 10000).sum() > 0:
        raise ValueError
    imwrite(str(dest), stack[::-1, :, :].astype(np.uint16), resolution=(1/0.6, 1/0.6))

In [ ]:
# Loop over suite2p folders and restructure the suite2p data into a
# 3D anatomy stack, a 3D ROI stack, and a h5 files with the cell traces.

master_path = Path(".../suite2p_batch200715")
paths = list(master_path.glob("*f[0-9]*"))

anatomy_path = master_path / "anatomies"
rois_path = master_path / "rois"
for p in [anatomy_path, rois_path]:
    p.mkdir(exist_ok=True)

for path in paths:
    traces = np.load(path / "F.npy")[()]
    iscell = np.load(path / "iscell.npy")
    ops = np.load(path / "ops.npy", allow_pickle=True)[()]
    stat = np.load(path / "stat.npy", allow_pickle=True)[()]

    rois_stack = np.zeros((ops["nplanes"],) + ops["refImg"].shape)
    
    # Place ROIs in ROI stack:
    k = 1
    i = 0
    plane = rois_stack[i, :, :]
    for j, a in enumerate(stat):
        ypix = a['ypix'].flatten()
        xpix = a['xpix'].flatten()
        if iscell[j, 0]:
            rois_stack[a["iplane"], ypix % rois_stack.shape[1], xpix % rois_stack.shape[2]] = k

            k += 1
            
    # Create anatomy stack:
    anatomy_stack = np.zeros((ops["nplanes"],) + ops["refImg"].shape)
    flat_img = ops["meanImg"]
    n = [m // r for m, r in zip(flat_img.shape, ops["refImg"].shape)]
    # print(n[0], n[1])
    for i_plane in range(ops["nplanes"]):
        # print(i_plane, (int(i_plane / n[1]), i_plane % n[1]))
        img_idxs = (i_plane // n[1], i_plane % n[1])
        anatomy_stack[i_plane, :, :] = flat_img[tuple([slice(idx*s, (idx + 1)*s) for idx, s in zip(img_idxs, anatomy_stack.shape[1:])])]
        

    save_anatomy(anatomy_path / f"{path.name}_anatomy.tif", anatomy_stack)
    save_rois(rois_path / f"{path.name}_roi.tif", rois_stack)
    # select traces:
    traces_sel = traces[iscell[:, 0].astype(np.int) > 0, :]        
    
    fl.save(master_path / f"exported_{path.name}.h5", dict(traces=traces_sel, stack=rois_stack), compression=None)
